In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=021096392db74f07c205d76d1db76bb31d752ebb0624613fa20643a5130127dd
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [36]:
import numpy as np

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .appName("Colab")\
        .getOrCreate()

spark

In [8]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe.select(col).dtypes[0][1] == "string"]
    num_but_cat = [col for col in dataframe.columns if len(dataframe.select(col).distinct().collect()) < cat_th and
                   dataframe.select(col).dtypes[0][1] != "string"]
    cat_but_car = [col for col in dataframe.columns if len(dataframe.select(col).distinct().collect()) > car_th and
                   dataframe.select(col).dtypes[0][1] == "string"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe.select(col).dtypes[0][1] != "string"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.count()}")
    print(f"Variables: { len(dataframe.columns)}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

In [6]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

df_pyspark = spark.read.csv("/content/train.csv", header = True, inferSchema = True) # second way to read
df_pyspark

Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NA,Attchd,2003,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NA,NA,NA,0,2,2008,WD,Normal,208500
2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NA,NA,NA,0,5,2007,WD,Normal,181500
3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NA,NA,NA,0,9,2008,WD,Normal,223500
4,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NA,NA,NA,0,12,2008,WD,Normal,250000
6,50,RL,85,14115,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NA,Attchd,1993,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NA,MnPrv,Shed,700,10,2009,WD,Normal,143000
7,20,RL,75,10084,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NA,NA,NA,0,8,2007,WD,Normal,307000
8,60,RL,NA,10382,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NA,NA,Shed,350,11,2009,WD,Normal,200000
9,50,RM,51,6120,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NA,NA,NA,0,4,2008,WD,Abnorml,129900
10,190,RL,50,7420,Pave,NA,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,GasA,Ex,Y,SBrkr,1077,0,0,1077,1,0,1,0,2,2,TA,5,Typ,2,TA,Attchd,1939,RFn,1,205,Gd,TA,Y,0,4,0,0,0,0,NA,NA,NA,0,1,2008,W

In [9]:
cat_cols, num_cols, cat_but_car = grab_col_names(df_pyspark)

Observations: 1460
Variables: 81
cat_cols: 53
num_cols: 24
cat_but_car: 4
num_but_cat: 11


In [40]:
# specify the strings to remove
strings_to_remove = ['GarageYrBlt', 'YrSold']

cat_cols = [x for x in cat_cols if x not in strings_to_remove]

In [ ]:
df_pyspark.select(["MSZoning", "BsmtCond", "OverallQual"]).show()

+--------+--------+-----------+
|MSZoning|BsmtCond|OverallQual|
+--------+--------+-----------+
|      RL|      TA|          7|
|      RL|      TA|          6|
|      RL|      TA|          7|
|      RL|      Gd|          7|
|      RL|      TA|          8|
|      RL|      TA|          5|
|      RL|      TA|          8|
|      RL|      TA|          7|
|      RM|      TA|          7|
|      RL|      TA|          5|
|      RL|      TA|          5|
|      RL|      TA|          9|
|      RL|      TA|          5|
|      RL|      TA|          7|
|      RL|      TA|          6|
|      RM|      TA|          7|
|      RL|      TA|          6|
|      RL|      NA|          4|
|      RL|      TA|          5|
|      RL|      TA|          5|
+--------+--------+-----------+
only showing top 20 rows



In [ ]:
df_pyspark.select("LotArea").agg({"LotArea": "avg"})

avg(LotArea)
10516.828082191782


In [ ]:
from pyspark.sql.functions import mean

df_pyspark.select(*[mean(c).alias(f"mean_of_{c}") for c in df_pyspark.columns])

mean_of_Id,mean_of_MSSubClass,mean_of_MSZoning,mean_of_LotFrontage,mean_of_LotArea,mean_of_Street,mean_of_Alley,mean_of_LotShape,mean_of_LandContour,mean_of_Utilities,mean_of_LotConfig,mean_of_LandSlope,mean_of_Neighborhood,mean_of_Condition1,mean_of_Condition2,mean_of_BldgType,mean_of_HouseStyle,mean_of_OverallQual,mean_of_OverallCond,mean_of_YearBuilt,mean_of_YearRemodAdd,mean_of_RoofStyle,mean_of_RoofMatl,mean_of_Exterior1st,mean_of_Exterior2nd,mean_of_MasVnrType,mean_of_MasVnrArea,mean_of_ExterQual,mean_of_ExterCond,mean_of_Foundation,mean_of_BsmtQual,mean_of_BsmtCond,mean_of_BsmtExposure,mean_of_BsmtFinType1,mean_of_BsmtFinSF1,mean_of_BsmtFinType2,mean_of_BsmtFinSF2,mean_of_BsmtUnfSF,mean_of_TotalBsmtSF,mean_of_Heating,mean_of_HeatingQC,mean_of_CentralAir,mean_of_Electrical,mean_of_1stFlrSF,mean_of_2ndFlrSF,mean_of_LowQualFinSF,mean_of_GrLivArea,mean_of_BsmtFullBath,mean_of_BsmtHalfBath,mean_of_FullBath,mean_of_HalfBath,mean_of_BedroomAbvGr,mean_of_KitchenAbvGr,mean_of_KitchenQual,mean_of_TotRmsAbvGrd,mean_of_Functional,mean_of_Fireplaces,mean_of_FireplaceQu,mean_of_GarageType,mean_of_GarageYrBlt,mean_of_GarageFinish,mean_of_GarageCars,mean_of_GarageArea,mean_of_GarageQual,mean_of_GarageCond,mean_of_PavedDrive,mean_of_WoodDeckSF,mean_of_OpenPorchSF,mean_of_EnclosedPorch,mean_of_3SsnPorch,mean_of_ScreenPorch,mean_of_PoolArea,mean_of_PoolQC,mean_of_Fence,mean_of_MiscFeature,mean_of_MiscVal,mean_of_MoSold,mean_of_YrSold,mean_of_SaleType,mean_of_SaleCondition,mean_of_SalePrice
730.5,56.897260273972606,null,70.04995836802665,10516.828082191782,null,null,null,null,null,null,null,null,null,null,null,null,6.0993150684931505,5.575342465753424,1971.267808219178,1984.8657534246574,null,null,null,null,null,103.68526170798899,null,null,null,null,null,null,null,443.6397260273973,null,46.54931506849315,567.2404109589041,1057.4294520547944,null,null,null,null,1162.626712328767,346.99246575342465,5.844520547945206,1515.463698630137,0.42534246575342466,0.057534246575342465,1.5650684931506849,0.38287671232876713,2.8664383561643834,1.0465753424657533,null,6.517808219178082,null,0.613013698630137,null,null,1978.5061638868744,null,1.7671232876712328,472.9801369863014,null,null,null,94.2445205479452,46.66027397260274,21.954109589041096,3.4095890410958902,15.060958904109588,2.758904109589041,null,null,null,43.489041095890414,6.321917808219178,2007.8157534246575,null,null,180921.19589041095


In [ ]:
df_pyspark.groupBy("LotShape").agg({col: "avg" for col in ["LotFrontage", "LotArea"]})

LotShape,avg(LotFrontage),avg(LotArea)
IR1,76.09463722397476,11894.545454545454
IR2,76.5,23733.658536585364
Reg,67.03877790834312,8876.915675675677
IR3,138.42857142857142,41338.2


In [12]:
df_pyspark = df_pyspark.withColumn("LotFrontage", df_pyspark["LotFrontage"].cast("int"))

In [13]:
df_pyspark = df_pyspark.na.drop()

In [14]:
from pyspark.sql.types import FloatType, IntegerType
from pyspark.sql.functions import udf

def squared(value):
    return value * value

squared_udf = udf(squared, IntegerType())

df_pyspark.withColumn("LotFrontage_squared", squared_udf("LotFrontage"))

Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_squared
1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NA,Attchd,2003,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NA,NA,NA,0,2,2008,WD,Normal,208500,4225
2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NA,NA,NA,0,5,2007,WD,Normal,181500,6400
3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NA,NA,NA,0,9,2008,WD,Normal,223500,4624
4,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000,3600
5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NA,NA,NA,0,12,2008,WD,Normal,250000,7056
6,50,RL,85,14115,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NA,Attchd,1993,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NA,MnPrv,Shed,700,10,2009,WD,Normal,143000,7225
7,20,RL,75,10084,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NA,NA,NA,0,8,2007,WD,Normal,307000,5625
9,50,RM,51,6120,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NA,NA,NA,0,4,2008,WD,Abnorml,129900,2601
10,190,RL,50,7420,Pave,NA,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,GasA,Ex,Y,SBrkr,1077,0,0,1077,1,0,1,0,2,2,TA,5,Typ,2,TA,Attchd,1939,RFn,1,205,Gd,TA,Y,0,4,0,0,0,0,NA,NA,NA,0,1,2008,WD,Normal,118000,2500
11,20,RL,70,11200,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,5,1965,1965,Hip,CompShg,HdBoard,HdBoard,None,0,TA,TA,CBlock,TA,TA,No,Rec,906,Unf,0,134,1040,GasA,Ex,Y,SBrkr,1040,0,0,1040,1,0,1,0,3,1,TA,5,Typ,0,NA,Detchd,1965,Un

In [ ]:
train, test = df_pyspark.randomSplit([0.7, 0.3])

train, test

In [59]:
numerical_features_lst = train[num_cols].columns
numerical_features_lst.remove('Id')
numerical_features_lst.remove('SalePrice')
# numerical_features_lst.remove('id')
# numerical_features_lst.remove('ocean_proximity')

numerical_features_lst

['MSSubClass',
 'LotArea',
 'OverallQual',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'TotRmsAbvGrd',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'MiscVal',
 'MoSold']

In [24]:
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols=numerical_features_lst,
                  outputCols=numerical_features_lst)

imputer = imputer.fit(train)

train = imputer.transform(train)
test = imputer.transform(test)

train

Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,numerical_features_lst
5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NA,NA,NA,0,12,2008,WD,Normal,250000,"[60.0,14260.0,8.0..."
7,20,RL,75,10084,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NA,NA,NA,0,8,2007,WD,Normal,307000,"[20.0,10084.0,8.0..."
10,190,RL,50,7420,Pave,NA,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,GasA,Ex,Y,SBrkr,1077,0,0,1077,1,0,1,0,2,2,TA,5,Typ,2,TA,Attchd,1939,RFn,1,205,Gd,TA,Y,0,4,0,0,0,0,NA,NA,NA,0,1,2008,WD,Normal,118000,"[190.0,7420.0,5.0..."
12,60,RL,85,11924,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,2Story,9,5,2005,2006,Hip,CompShg,WdShing,Wd Shng,Stone,286,Ex,TA,PConc,Ex,TA,No,GLQ,998,Unf,0,177,1175,GasA,Ex,Y,SBrkr,1182,1142,0,2324,1,0,3,0,4,1,Ex,11,Typ,2,Gd,BuiltIn,2005,Fin,3,736,TA,TA,Y,147,21,0,0,0,0,NA,NA,NA,0,7,2006,New,Partial,345000,"[60.0,11924.0,9.0..."
16,45,RM,51,6120,Pave,NA,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Norm,Norm,1Fam,1.5Unf,7,8,1929,2001,Gable,CompShg,Wd Sdng,Wd Sdng,None,0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,832,832,GasA,Ex,Y,FuseA,854,0,0,854,0,0,1,0,2,1,TA,5,Typ,0,NA,Detchd,1991,Unf,2,576,TA,TA,Y,48,112,0,0,0,0,NA,GdPrv,NA,0,7,2007,WD,Normal,132000,"[45.0,6120.0,7.0,..."
18,90,RL,72,10791,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,Duplex,1Story,4,5,1967,1967,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,Slab,NA,NA,NA,NA,0,NA,0,0,0,GasA,TA,Y,SBrkr,1296,0,0,1296,0,0,2,0,2,2,TA,6,Typ,0,NA,CarPort,1967,Unf,2,516,TA,TA,Y,0,0,0,0,0,0,NA,NA,Shed,500,10,2006,WD,Normal,90000,"(23,[0,1,2,3,4,9,..."
19,20,RL,66,13695,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,SawyerW,RRAe,Norm,1Fam,1Story,5,5,2004,2004,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,PConc,TA,TA,No,GLQ,646,Unf,0,468,1114,GasA,Ex,Y,SBrkr,1114,0,0,1114,1,0,1,1,3,1,Gd,6,Typ,0,NA,Detchd,2004,Unf,2,576,TA,TA,Y,0,102,0,0,0,0,NA,NA,NA,0,6,2008,WD,Normal,159000,"[20.0,13695.0,5.0..."
21,60,RL,101,14215,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,NridgHt,Norm,Norm,1Fam,2Story,8,5,2005,2006,Gable,CompShg,VinylSd,VinylSd,BrkFace,380,Gd,TA,PConc,Ex,TA,Av,Unf,0,Unf,0,1158,1158,GasA,Ex,Y,SBrkr,1158,1218,0,2376,0,0,3,1,4,1,Gd,9,Typ,1,Gd,BuiltIn,2005,RFn,3,853,TA,TA,Y,240,154,0,0,0,0,NA,NA,NA,0,11,2006,New,Partial,325300,"[60.0,14215.0,8.0..."
24,120,RM,44,4224,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,1Story,5,7,1976,1976,Gable,CompShg,CemntBd,CmentBd,None,0,TA,TA,PConc,Gd,TA,No,GLQ,840,Unf,0,200,1040,GasA,TA,Y,SBrkr,1060,0,0,1060,1,0,1,0,3,1,TA,6,Typ,1,TA,Attchd,1976,Unf,2,572,TA,TA,Y,100,110,0,0,0,0,NA,NA,NA,0,6,2007,WD,Normal,129900,"[120.0,4224.0,5.0..."
26,20,RL,110,14230,Pave,NA,Reg,Lvl,AllPub,Corner,Gtl,NridgHt,Norm,Norm,1F

In [60]:
from pyspark.ml.feature import VectorAssembler

numerical_vector_assembler = VectorAssembler(inputCols=numerical_features_lst,
                                              outputCol="numerical_features_lst")

train = numerical_vector_assembler.transform(train)
test = numerical_vector_assembler.transform(test)

train

Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,numerical_features_lst
1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NA,Attchd,2003,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NA,NA,NA,0,2,2008,WD,Normal,208500,"[60.0,8450.0,7.0,..."
2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NA,NA,NA,0,5,2007,WD,Normal,181500,"[20.0,9600.0,6.0,..."
5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NA,NA,NA,0,12,2008,WD,Normal,250000,"[60.0,14260.0,8.0..."
6,50,RL,85,14115,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NA,Attchd,1993,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NA,MnPrv,Shed,700,10,2009,WD,Normal,143000,"[50.0,14115.0,5.0..."
7,20,RL,75,10084,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NA,NA,NA,0,8,2007,WD,Normal,307000,"[20.0,10084.0,8.0..."
10,190,RL,50,7420,Pave,NA,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,GasA,Ex,Y,SBrkr,1077,0,0,1077,1,0,1,0,2,2,TA,5,Typ,2,TA,Attchd,1939,RFn,1,205,Gd,TA,Y,0,4,0,0,0,0,NA,NA,NA,0,1,2008,WD,Normal,118000,"[190.0,7420.0,5.0..."
11,20,RL,70,11200,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,5,1965,1965,Hip,CompShg,HdBoard,HdBoard,None,0,TA,TA,CBlock,TA,TA,No,Rec,906,Unf,0,134,1040,GasA,Ex,Y,SBrkr,1040,0,0,1040,1,0,1,0,3,1,TA,5,Typ,0,NA,Detchd,1965,Unf,1,384,TA,TA,Y,0,0,0,0,0,0,NA,NA,NA,0,2,2008,WD,Normal,129500,"(22,[0,1,2,3,4,5,..."
12,60,RL,85,11924,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,2Story,9,5,2005,2006,Hip,CompShg,WdShing,Wd Shng,Stone,286,Ex,TA,PConc,Ex,TA,No,GLQ,998,Unf,0,177,1175,GasA,Ex,Y,SBrkr,1182,1142,0,2324,1,0,3,0,4,1,Ex,11,Typ,2,Gd,BuiltIn,2005,Fin,3,736,TA,TA,Y,147,21,0,0,0,0,NA,NA,NA,0,7,2006,New,Partial,345000,"[60.0,11924.0,9.0..."
14,20,RL,91,10652,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,7,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,Stone,306,Gd,TA,PConc,Gd,TA,Av,Unf,0,Unf,0,1494,1494,GasA,Ex,Y,SBrkr,1494,0,0,1494,0,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2006,RFn,3,840,TA,TA,Y,160,33,0,0,0,0,NA,NA,NA,0,8,2007,New,Partial,279500,"[20.0,10652.0,7.0..."
16,45,RM,51,6120,Pave,NA,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Norm,Norm,1Fam,1.5

In [29]:
train.select("numerical_features_lst").take(2)

[Row(numerical_features_lst=DenseVector([60.0, 8450.0, 7.0, 2003.0, 2003.0, 706.0, 0.0, 150.0, 856.0, 856.0, 854.0, 0.0, 1710.0, 8.0, 548.0, 0.0, 61.0, 0.0, 0.0, 0.0, 0.0, 2.0])),
 Row(numerical_features_lst=DenseVector([70.0, 9550.0, 7.0, 1915.0, 1970.0, 216.0, 0.0, 540.0, 756.0, 961.0, 756.0, 0.0, 1717.0, 7.0, 642.0, 0.0, 35.0, 272.0, 0.0, 0.0, 0.0, 2.0]))]

In [61]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol='numerical_features_lst',
                        outputCol='scaled_numerical_feature_vector',
                        withStd=True, withMean=True)

scaler = scaler.fit(train)

train = scaler.transform(train)
test = scaler.transform(test)

train.show(3)

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+----------------------+-------------------------------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilitie

In [31]:
train.select('scaled_numerical_feature_vector').take(3)

[Row(scaled_numerical_feature_vector=DenseVector([0.0861, -0.1784, 0.6086, 1.001, 0.8195, 0.6612, -0.2649, -0.9715, -0.4495, -0.7965, 1.164, -0.1289, 0.3832, 0.8849, 0.3394, -0.7592, 0.1794, -0.3818, -0.0988, -0.2533, -0.1332, -1.6062])),
 Row(scaled_numerical_feature_vector=DenseVector([0.3245, -0.0519, 0.6086, -1.7325, -0.724, -0.4514, -0.2649, -0.1178, -0.6801, -0.5191, 0.9397, -0.1289, 0.3966, 0.2827, 0.7749, -0.7592, -0.1916, 4.1112, -0.0988, -0.2533, -0.1332, -1.6062])),
 Row(scaled_numerical_feature_vector=DenseVector([0.0861, 0.4897, 1.3029, 0.9079, 0.6792, 0.5454, -0.2649, -0.2272, 0.2169, -0.033, 1.6193, -0.1289, 1.3169, 1.487, 1.6738, 0.8869, 0.5076, -0.3818, -0.0988, -0.2533, -0.1332, 2.0754]))]

In [63]:
cat_cols = [col for col in cat_cols if train.select(col).distinct().count() != 1]

In [64]:
from pyspark.ml.feature import StringIndexer

output_cols = [col + '_indexed' for col in cat_cols]

indexer = StringIndexer(inputCols=cat_cols,
                        outputCols=output_cols)

indexer = indexer.fit(train)
train = indexer.transform(train)
test = indexer.transform(test)

train.show(3)

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+----------------------+-------------------------------+----------------+--------------+-------------+----------------+-------------------+-----

In [46]:
set(train.select("SaleCondition_indexed").collect())

{Row(SaleCondition_indexed=0.0),
 Row(SaleCondition_indexed=1.0),
 Row(SaleCondition_indexed=2.0),
 Row(SaleCondition_indexed=3.0),
 Row(SaleCondition_indexed=4.0),
 Row(SaleCondition_indexed=5.0)}

In [ ]:
set(train.select("Utilities_indexed").collect())

In [56]:
train.select("Utilities_indexed").distinct().count() == 1

True

In [66]:
from pyspark.ml.feature import OneHotEncoder

output_cols_ohe = [col + '_ohe' for col in output_cols]

train = train.drop(*cat_cols)
test = train.drop(*cat_cols)

one_hot_encoder = OneHotEncoder(inputCols=output_cols,
                                outputCols=output_cols_ohe)


one_hot_encoder = one_hot_encoder.fit(train)

train = one_hot_encoder.transform(train)
test = one_hot_encoder.transform(test)

train = train.drop(*output_cols)
test = train.drop(*output_cols)

train.show(3)

+---+----------+-----------+-------+---------+------------+-----------+---------+------------+----------+----------+----------+---------+-----------+--------+--------+------------+---------+------------+-----------+----------+----------+-----------+-------------+---------+-----------+-------+------+------+---------+----------------------+-------------------------------+--------------------+------------------+-----------------+--------------------+-----------------------+---------------------+---------------------+----------------------+----------------------+--------------------+----------------------+---------------------+--------------------+-----------------------+-----------------------+----------------------+---------------------+---------------------+----------------------+--------------------+--------------------+------------------------+------------------------+------------------------+-------------------+---------------------+----------------------+----------------------+-----

In [77]:
input_cols_pp = [col for col in output_cols_ohe]
input_cols_pp.append("scaled_numerical_feature_vector")

In [81]:
assembler = VectorAssembler(inputCols= input_cols_pp,
                            outputCol='final_feature_vector')

train = assembler.transform(train)
test = assembler.transform(test)

In [83]:
train.select('final_feature_vector').take(2)

[Row(final_feature_vector=SparseVector(247, {0: 1.0, 4: 1.0, 5: 1.0, 7: 1.0, 10: 1.0, 13: 1.0, 17: 1.0, 19: 1.0, 27: 1.0, 32: 1.0, 37: 1.0, 43: 1.0, 47: 1.0, 51: 1.0, 64: 1.0, 79: 1.0, 83: 1.0, 85: 1.0, 89: 1.0, 95: 1.0, 98: 1.0, 102: 1.0, 107: 1.0, 112: 1.0, 118: 1.0, 122: 1.0, 125: 1.0, 126: 1.0, 132: 1.0, 134: 1.0, 139: 1.0, 144: 1.0, 151: 1.0, 153: 1.0, 158: 1.0, 163: 1.0, 165: 1.0, 168: 1.0, 172: 1.0, 175: 1.0, 183: 1.0, 188: 1.0, 196: 1.0, 197: 1.0, 199: 1.0, 203: 1.0, 204: 1.0, 211: 1.0, 214: 1.0, 217: 1.0, 221: 1.0, 225: 0.0342, 226: -0.1763, 227: 0.6265, 228: 1.019, 229: 0.8585, 230: 0.6557, 231: -0.2656, 232: -0.9853, 233: -0.4707, 234: -0.7804, 235: 1.1166, 236: -0.1401, 237: 0.3652, 238: 0.9156, 239: 0.3453, 240: -0.7441, 241: 0.2192, 242: -0.3756, 243: -0.0853, 244: -0.2573, 245: -0.14, 246: -1.6174})),
 Row(final_feature_vector=SparseVector(247, {0: 1.0, 4: 1.0, 5: 1.0, 7: 1.0, 10: 1.0, 16: 1.0, 17: 1.0, 20: 1.0, 27: 1.0, 32: 1.0, 36: 1.0, 43: 1.0, 47: 1.0, 52: 1.0, 65: 1

In [85]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='final_feature_vector',
                      labelCol='SalePrice')

lr

LinearRegression_b9d3fe17b8fd

In [86]:
lr = lr.fit(train)

lr

LinearRegressionModel: uid=LinearRegression_b9d3fe17b8fd, numFeatures=247

In [87]:
pred_train_df = lr.transform(train).withColumnRenamed('prediction',
                                                      'predicted_sale_price')

pred_train_df.show(5)

+---+----------+-----------+-------+---------+------------+-----------+---------+------------+----------+----------+----------+---------+-----------+--------+--------+------------+---------+------------+-----------+----------+----------+-----------+-------------+---------+-----------+-------+------+------+---------+----------------------+-------------------------------+--------------------+------------------+-----------------+--------------------+-----------------------+---------------------+---------------------+----------------------+----------------------+--------------------+----------------------+---------------------+--------------------+-----------------------+-----------------------+----------------------+---------------------+---------------------+----------------------+--------------------+--------------------+------------------------+------------------------+------------------------+-------------------+---------------------+----------------------+----------------------+-----

In [88]:
pred_test_df = lr.transform(test).withColumnRenamed('prediction', 'predicted_sale_price')

pred_test_df.show(5)

+---+----------+-----------+-------+---------+------------+-----------+---------+------------+----------+----------+----------+---------+-----------+--------+--------+------------+---------+------------+-----------+----------+----------+-----------+-------------+---------+-----------+-------+------+------+---------+----------------------+-------------------------------+--------------------+------------------+-----------------+--------------------+-----------------------+---------------------+---------------------+----------------------+----------------------+--------------------+----------------------+---------------------+--------------------+-----------------------+-----------------------+----------------------+---------------------+---------------------+----------------------+--------------------+--------------------+------------------------+------------------------+------------------------+-------------------+---------------------+----------------------+----------------------+-----

In [90]:
pred_test_pd_df = pred_test_df.toPandas()

pred_test_pd_df.head(2)

,Id,MSSubClass,LotFrontage,LotArea,Utilities,Neighborhood,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,...,BsmtHalfBath_indexed_ohe,FullBath_indexed_ohe,HalfBath_indexed_ohe,BedroomAbvGr_indexed_ohe,KitchenAbvGr_indexed_ohe,Fireplaces_indexed_ohe,GarageCars_indexed_ohe,PoolArea_indexed_ohe,final_feature_vector,predicted_sale_price
0,1,60,65,8450,AllPub,CollgCr,7,2003,2003,196,...,"(1.0, 0.0)","(1.0, 0.0, 0.0)","(0.0, 1.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, ...",208456.986086
1,2,20,80,9600,AllPub,Veenker,6,1976,1976,0,...,"(0.0, 1.0)","(1.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, ...",187626.987151


In [91]:
predictions_and_actuals = pred_test_df[['predicted_sale_price',
                                        'SalePrice']]
                                    
predictions_and_actuals_rdd = predictions_and_actuals.rdd

predictions_and_actuals_rdd.take(2)

[Row(predicted_sale_price=208456.98608612228, SalePrice=208500),
 Row(predicted_sale_price=187626.9871505, SalePrice=181500)]

In [92]:
predictions_and_actuals_rdd = predictions_and_actuals_rdd.map(tuple)

predictions_and_actuals_rdd.take(2)

[(208456.98608612228, 208500), (187626.9871505, 181500)]

In [98]:
# from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator()

evaluator.setLabelCol("SalePrice")
evaluator.setPredictionCol("predicted_sale_price")

mse = evaluator.evaluate(predictions_and_actuals, {evaluator.metricName: "mse"})
rmse = evaluator.evaluate(predictions_and_actuals, {evaluator.metricName: "rmse"})
r2 = evaluator.evaluate(predictions_and_actuals, {evaluator.metricName: "r2"})

s = '''
Mean Squared Error:      {0}
Root Mean Squared Error: {1}
R**2:                    {2}
'''.format(mse,
           rmse,
           r2
           )

print(s)


Mean Squared Error:      380896254.0126318
Root Mean Squared Error: 19516.563581036284
R**2:                    0.9479908968530074

